## QUIÉN ES QUIÉN EN LOS PRECIOS

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc

In [3]:
spark = SparkSession.builder.master("local[1]").appName("Precios").getOrCreate()

In [4]:
df = spark.read.option("header",True).csv("all_data.csv")
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [5]:
df.show()

+--------------------+--------------------+--------------------+--------------------+-----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|               marca|           categoria|         catalogo|precio|       fechaRegistro|   cadenaComercial|                giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------------------+--------------------+-----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|            ESTRELLA|    MATERIAL ESCOLAR| UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|          PAPELERIAS|ABASTECEDORA 

1. ¿Cuántos registros hay?
2. ¿Cuántas categorías?
3. ¿Cuántas cadenas comerciales están siendo monitoreadas?

In [6]:
print(df.count())

62530715


In [18]:
from pyspark.sql.types import *

In [21]:
spark.sql("DROP TABLE IF EXISTS precios")

DataFrame[]

In [23]:
df.createOrReplaceTempView("precios")
spark.sql("select * from precios").show(n=5)

+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.125417|
|            CRAYONE

#### Vemos algunas formas de calcular usando sql

In [33]:
spark.sql('SELECT COUNT ( DISTINCT categoria ) FROM precios').show()

+-------------------------+
|count(DISTINCT categoria)|
+-------------------------+
|                       41|
+-------------------------+



In [ ]:
#spark.sql("select categoria from precios").groupBy('categoria').count().show()

In [30]:
c=  df.select(countDistinct("categoria"))

In [ ]:
c.show()

In [34]:
e = df.select(countDistinct('cadenaComercial'))

In [ ]:
e.show()

¿Cuáles son los productos más monitoreados en cada entidad?

¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

Toma mucho tiempo asi que probaremos primero las consultas con el .1% de los datos que son 62 mil registros aproximadamente.

In [37]:
df1 = df.sample(0.001, seed=2021).collect()

In [39]:
schema1 = df.schema

In [42]:
dfs = spark.createDataFrame(df1,schema1)
print(dfs.schema)
dfs.show()

StructType(List(StructField(producto,StringType,true),StructField(presentacion,StringType,true),StructField(marca,StringType,true),StructField(categoria,StringType,true),StructField(catalogo,StringType,true),StructField(precio,StringType,true),StructField(fechaRegistro,StringType,true),StructField(cadenaComercial,StringType,true),StructField(giro,StringType,true),StructField(nombreComercial,StringType,true),StructField(direccion,StringType,true),StructField(estado,StringType,true),StructField(municipio,StringType,true),StructField(latitud,StringType,true),StructField(longitud,StringType,true)))
+--------------------+--------------------+--------------------+--------------------+-----------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+
|            producto|        presentacion|               marca|           categoria|         catalogo|precio|       f

In [43]:
dfs.createOrReplaceTempView("precios1")
spark.sql("select * from precios1").show(n=5)

+--------------------+--------------------+--------------------+--------------------+-----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------+--------------------+---------+-----------+
|            producto|        presentacion|               marca|           categoria|         catalogo|precio|       fechaRegistro|   cadenaComercial|                giro|     nombreComercial|           direccion|    estado|           municipio|  latitud|   longitud|
+--------------------+--------------------+--------------------+--------------------+-----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------+--------------------+---------+-----------+
|REPUESTOS DE NAVAJAS|ESTUCHE CON 2. MA...|            GILLETTE|ARTS. PARA EL CUI...|          BASICOS| 74.62|2011-01-10 00:00:...|COMERCIAL MEXICANA|TIENDA DE AUTOSER...|COMERCIAL MEXICAN...|AV. 

In [44]:
spark.sql("select producto from precios1").groupBy('producto').count().show()

+--------------------+-----+
|            producto|count|
+--------------------+-----+
|             LUTORAL|    7|
|              VARIOS|  386|
|             GYNOVIN|    6|
|            KAOMYCIN|    5|
|           BEDOYECTA|   24|
|              PIERNA|    1|
|        PRAVASTATINA|   11|
|    SAN RAFAEL. REAL|  121|
|            FRIJOLES|  111|
|             FOLDERS|   25|
|    ACEITUNA. GORDAL|   20|
|            GARBANZA|   44|
|       FILETE DE RES|    2|
|CAFE TOSTADO Y MO...|  198|
|            MYCOSPOR|   10|
|              COMPAS|   36|
|            AMARYL M|   22|
|    CALCIGENOL DOBLE|    5|
|           GRANEODIN|    5|
|            GAMMADOL|    7|
+--------------------+-----+
only showing top 20 rows



In [56]:
# ¿Cuáles son los productos más monitoreados en cada entidad?
spark.sql("select producto, estado from precios1 where estado is not null").groupBy('estado','producto').count().orderBy('estado').show(5)

+--------------------+---------+-----+
|              estado| producto|count|
+--------------------+---------+-----+
| COL. EDUARDO GUERRA|SINGULAIR|    1|
| COL. EDUARDO GUERRA|ESMERALDA|    1|
| COL. EDUARDO GUERRA|      KIR|    1|
| COL. EDUARDO GUERRA|MERMELADA|    1|
| COL. EDUARDO GUERRA|  SHAMPOO|    1|
+--------------------+---------+-----+
only showing top 5 rows



In [50]:
spark.sql("select producto, cadenaComercial from precios1").groupBy('cadenaComercial').count().orderBy(desc('count')).show(5)

+--------------------+-----+
|     cadenaComercial|count|
+--------------------+-----+
|            WAL-MART| 8520|
|      BODEGA AURRERA| 6771|
|             SORIANA| 6743|
|MEGA COMERCIAL ME...| 4914|
|            CHEDRAUI| 4308|
+--------------------+-----+
only showing top 5 rows



In [57]:
spark.sql('SELECT year(fechaRegistro) as anio, avg(precio) as sales FROM precios1 GROUP BY year(fechaRegistro) ORDER BY anio').show()

+----+-----------------+
|anio|            sales|
+----+-----------------+
|2011| 467.377692504144|
|2012|504.8278437633048|
|2013|455.2684965449124|
|2014|491.5235376448037|
|2015| 573.692171617195|
|2016|564.7799196344075|
+----+-----------------+



a. Genera una canasta de productos básicos que te permita comparar los precios
geográfica y temporalmente. Justifica tu elección y procedimiento.

> Tomamos los productos distintos vendidos por municipio.
Necesitamos productos que se vendan en al menos un porcentaje del total de municipios. Por ejemplo, considerar los primeros 20 articulos que se venden en al menos 80% de los municipios y ver si el resultado tiene suficientes ejemplos.


b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

> Revisar el precio promedio de cada producto por municipio.


c. ¿Hay algún patrón estacional entre años?

> Necesitamos revisar el cambio de precio de cada producto en el tiempo.
%sql
SELECT
year(fechaRegistro) as anio, 
avg(precio) as sales
FROM precios1
GROUP BY year(fechaRegistro)
ORDER BY anio;

>%sql
SELECT 
TRUNC(fechaRegistro, 'MM') as mes,
avg(precio) as precio
FROM precios1
GROUP BY TRUNC(fechaRegistro, 'MM')
ORDER BY mes;
Pero revisando los resultados por municipio e incluso por porducto de la canasta definida anteriormente.

d. ¿Cuál es el estado más caro y en qué mes?

> Podemos sacarlo de lo anterior.

e. ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con
estos datos?

- nulos generan problemas en los que podemos dejar con muchos faltantes en los productos o en las entidades.
- posiblemente los productos no aparecen en una lista en una entidad solo por falta en su compra. No es homogenea la compra de productos para verificar los precios.
- hay un tema de sesgo porque las compras fueron en tiendas de autoservicio (hasta donde recuerdo, falta verificar información)
- habría que buscar formas de realizar el análisis de manera eficiente.